# **Soni - do**
## **Generating Music with Machine Learning**


#### Author: Sonia Cobo
#### Date: July 2021

### Though this project doesn't have a hypothesis per se, it was done to kind off prove how AI has advanced and it is now able to generate music which has been associated with emotions and human capabilities for a long period of time.

In [12]:
# data augmentation - dividir canciones, modificarlas para tener mas datos

# Data

### The input to the model will be a series of notes from a MIDI file. MIDI (Musical Instrument Digital Interface) is a technical standard that describes a communications protocol, digital interface, and electrical connectors that connect a wide variety of electronic musical instruments and computers. They don't contain actual audio data and are small in size. They explain what notes are played, when they're played, and how long or loud each note should be.

### To keep the project simple only files with one instrument were chosen, in this case the instrument is piano and the type of songs is classical. 
### These songs have been obtained from the following datasets: http://www.piano-midi.de/ and https://www.mfiles.co.uk/classical-midi.htm


In [13]:
# no descargardas aun: https://github.com/Skuldur/Classical-Piano-Composer/tree/master/midi_songs
# https://drive.google.com/file/d/1qnQVK17DNVkU19MgVA4Vg88zRDvwCRXw/view

### Import all libraries

In [254]:
# data manipulation
import numpy as np
import pandas as pd 

# manipulate midi files
import glob
from music21 import *
#from music21 import converter, instrument, note, chord, meter, stream, duration, corpus
import pygame

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# route files
import os
import sys

# ml model
import pickle

import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional

# my libraries
import utils.mining_data_tb as md
#from utils.folders_tb import convert_to_json, save_json, read_json, dataframe_to_json
#import utils.visualization_tb as vis
#from utils.mysql_driver import MySQL


In [15]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7525799484492965909
]


### Paths

In [50]:
# The route of this file is added to the sys path to be able to import/export functions
sep = os.sep
def route (steps):
    """
    This function appends the route of the file to the sys path
    to be able to import files from/to other foders within the EDA project folder.
    """
    route = os.getcwd()
    for i in range(steps):
        route = os.path.dirname(route)
    sys.path.append(route)
    return route

In [54]:
# paths

# path to raw data
path = route(1) + sep + "data" + sep + "raw_data" + sep
# path to data in the right key
path_1 = route(1) + sep + "data" + sep + "converted_data" + sep
# path to compiled notes list
path_2 = route(1) + sep + "data" + sep + "notes" + sep
# path to generated models
path_3 = route(1) + sep + "models" + sep
# path to generated midi files
path_4 = route(1) + sep + "reports" + sep

## Midi file exploration

Hablar de frecuencia y la transpuesta de fourier

In [243]:
# All information from the midi file (i.e. notes, pitch, chord, time signature, etc) is contained within the component list
def info_midi (path):
    """
    It returns all midi file information given its path

    """
    file = converter.parse(path)
    components = []
    for element in file.recurse():  
        components.append(element)
    return components

components = info_midi(path + "alb_esp1.mid")
#components

### Now that the midi file has been studied and its structure is known, data will be split into two object types: notes, rests and chords. 

### Note objects contain information about the pitch, octave, and offset of the note.
### Pitch refers to the frequency of the sound, or how high or low it is and is represented with the letters [A, B, C, D, E, F, G].
### Octave refers to which set of pitches you use on a piano.
### Offset refers to where the note is located in the piece.
### Rests are the silences in the piece.
### Chord objects are a set of notes that are played at the same time.


### Songs are transposed into C major and A minor key to ease predictions

## Data preparation

### Relevant information from midi file is encoded and saved into an array.

### We append the pitch of every note object using its string notation since the most significant parts of the note can be recreated using the string notation of the pitch. And we append every chord by encoding the id of every note in the chord together into a single string, with each note being separated by a dot. 

In [319]:
# Each midi file contains notes and chords. These two properties will be the input and output of the LSTM network so 
# they need to be taken out from all midi files. 

def get_notes_per_song(path, filename):
    """
    This function extracts all the notes, rests and chords from one midi file
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file, filename (str)
    """
    components = info_midi(path + filename)
    note_list = []
    
    for element in components:
        # note pitches are extracted
        if isinstance(element, note.Note):
            note_list.append(str(element.pitch))
        # chords are extracted
        elif isinstance(element, chord.Chord):
            note_list.append(".".join(str(n) for n in element.normalOrder))    
        # rests are extracted
        elif isinstance(element, note.Rest):
            note_list.append("NULL")    #further transformation needs this value as str rather than np.nan

    with open(path_2 + "notes_chopin", "wb") as filepath:
        pickle.dump(note_list, filepath)
    
    return note_list

In [317]:
#note_list = get_notes_per_song(path_1, ".mid")

In [316]:
path

'c:\\Users\\Usuario\\Desktop\\The_Bridge\\ML_Project\\data\\raw_data\\'

In [320]:
def get_all_notes(path=path_1):
    """
    This function extracts all the notes, rests and chords from all midi files 
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file     
    """
    list_path = os.listdir(path)
    for elem in list_path:
        output = get_notes_per_song(path, elem)
    return output

note_list = get_all_notes()

In [321]:
len(note_list)

5354

In [322]:
# Load notes and chords previously separated
def load_notes (path, filename):
    """
    Load the note list containing pitches, rests and chords.
    
    Param: Path of the saved note list, and its name as string
    """
    with open(path + filename, "rb") as f:
        loaded_notes = pickle.load(f)
        return loaded_notes

note_list = load_notes(path_2, "notes_chopin")
#note_list

### The model will be first trained with a small proportioned of the songs to expedite time. Once the model is tunned properly all songs will be passed to improve its training.

### Now that all notes, rests and chords are in a list, these will be transformed from categorical data to integer-based numerical data. It is necessary to create input sequences for the network and their respective outputs. The output for each input sequence will be the first note or chord that comes after the sequence of notes in the input sequence in our list of notes.

In [323]:
def prepare_sequences(notes, sequence_length, step):
    """ 
    Prepare the sequences used by the neural network 

    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    #sequence_in = []
    #sequence_out = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, step):    
        network_input.append(notes[i:i + sequence_length])
        network_output.append(notes[i + sequence_length])
        # exchange their values for their integer-code

        # network_input.append([note_to_int[elem] for elem in sequence_in])
        # network_output.append(note_to_int[sequence_out])

    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), len(pitchnames)))
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
        y[i, note_to_int[network_output[i]]] = 1

    return x, y

### The length of each sequence will be 100 notes/chords for now. This means that to predict the next note in the sequence the network has the previous 100 notes to help make the prediction

In [324]:
x, y = prepare_sequences(notes=note_list, sequence_length=100, step=3)  # length y step pueden variar  


Total unique notes: 169


In [325]:
print(x.shape)
print(y.shape)

(1752, 100, 169)
(1752, 169)


# Creation of the model

There are four different types of layers:

LSTM layers is a Recurrent Neural Net layer that takes a sequence as an input and can return either sequences (return_sequences=True) or a matrix.

Dropout layers are a regularisation technique that consists of setting a fraction of input units to 0 at each update during the training to prevent overfitting. The fraction is determined by the parameter used with the layer.

Dense layers or fully connected layers is a fully connected neural network layer where each input node is connected to each output node.

The Activation layer determines what activation function our neural network will use to calculate the output of a node.

In [326]:
def create_network():

    model = Sequential()
    model.add(LSTM(512, input_shape=(x.shape[1],x.shape[2])))
    model.add(Dense(x.shape[2]))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

    return model

In [327]:
model = create_network()

In [328]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 512)               1396736   
_________________________________________________________________
dense_6 (Dense)              (None, 169)               86697     
_________________________________________________________________
activation_6 (Activation)    (None, 169)               0         
Total params: 1,483,433
Trainable params: 1,483,433
Non-trainable params: 0
_________________________________________________________________


In [329]:
model.fit(x, y, epochs=10)#, batch_size=128)

Epoch 1/10
55/55 [==============================] - 27s 464ms/step - loss: 4.5829
Epoch 2/10
55/55 [==============================] - 25s 458ms/step - loss: 4.0160
Epoch 3/10
55/55 [==============================] - 25s 458ms/step - loss: 3.7334
Epoch 4/10
55/55 [==============================] - 25s 460ms/step - loss: 3.4617
Epoch 5/10
55/55 [==============================] - 25s 458ms/step - loss: 3.4128
Epoch 6/10
55/55 [==============================] - 26s 464ms/step - loss: 3.3100
Epoch 7/10
55/55 [==============================] - 25s 461ms/step - loss: 3.1478
Epoch 8/10
55/55 [==============================] - 25s 463ms/step - loss: 2.7769
Epoch 9/10
55/55 [==============================] - 25s 454ms/step - loss: 2.6104
Epoch 10/10
55/55 [==============================] - 25s 451ms/step - loss: 2.2827


In [330]:
# save the model
model.save(path_3 + "model_6_chopin.h5")

In [331]:
# load the model 
model_6 = tf.keras.models.load_model(path_3 + "model_6_chopin.h5")


In [332]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [340]:
def generate_notes(model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(note_list)-100-1)

    pitchnames = sorted(set(note_list))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = note_list[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music
    for note_index in range(100):
        prediction_input = np.zeros((1, 100, len(pitchnames)))
        for j, note in enumerate(pattern):
            prediction_input[0, j, note_to_int[note]] = 1.0
        preds = model.predict(prediction_input, verbose=0)[0]   #[0]?
        next_index = sample(preds, temperature=temperature)
        next_note = int_to_note[next_index]

        pattern = pattern[1:]
        pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns, next_index, pattern

In [341]:
prediction_output, patterns, next_index, pattern = generate_notes(model_6, temperature=1.0)
print(prediction_output)

['C4', 'B3', 'A3', 'F#3', 'A3', 'C4', 'C4', 'F3', 'D4', 'D4', 'C4', 'G3', 'C4', 'D4', 'F3', 'D4', 'A3', 'C4', 'B3', 'D4', 'C4', 'D4', 'B3', 'C4', 'D4', 'D4', '2', 'D4', 'B3', 'B3', 'D4', 'B3', 'D4', 'C4', 'D4', 'D4', 'E-1', 'D4', 'B-2', 'D3', 'D4', 'NULL', 'B-3', 'D4', 'B3', 'D4', 'B3', 'D4', 'D4', 'D4', 'NULL', 'D4', '2.5.7', 'D4', 'C4', 'D4', 'C4', 'C4', 'D4', 'D4', 'C7', 'D4', 'D4', 'C4', 'D4', 'D4', 'D4', 'C4', 'D4', 'C4', 'D4', 'D4', 'C4', 'C4', 'D4', 'B3', 'D4', 'D4', 'B3', 'D4', 'D4', 'B3', 'D4', 'D4', 'D4', 'C4', 'D4', 'B3', 'D4', 'D4', 'D4', 'B3', 'D4', 'D4', 'D3', 'D4', 'B3', 'B3', 'D4', 'B3']


# Output

In [342]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_11_chopin.mid")   # first output 01/07/2021

    return midi_stream

In [343]:
create_midi = create_midi(prediction_output, patterns, path_4)


In [250]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [252]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_11_chopin.mid")